# Script version 3.0.0 for Machine Learning Core (MLC)

## General settings

In [ ]:
import datetime
import logging
import os

from mlc_configurator import (
    MLCFilter,
    MLCFeature,
    arff_generator,
    ucf_generator,
    h_generator,
    get_devices,
    get_mlc_odr,
    get_mlc_input_type,
    get_mlc_inputs,
    get_accelerometer_odr,
    get_accelerometer_fs,
    get_gyroscope_odr,
    get_gyroscope_fs,
    get_filter_names,
    get_feature_names,
)
from decision_tree_generator import generate_decision_tree, generate_arff_subset
from mlc_test import test_arff_on_decisiontree
from mlc_script_log import Logger

In [2]:
current_directory = os.path.join(
    os.getcwd(),
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
os.makedirs(current_directory)
logging.info("Current directory: " + current_directory)
logger = Logger(current_directory, "")

arff_filename = os.path.join(current_directory, "features.arff")
ucf_filename = os.path.join(current_directory, "mlc.ucf")
h_filename = os.path.join(current_directory, "mlc.h")

## Step 1: Load log files, label data, define decision tree results
In this step data are loaded and results are assigned. 
Decision tree results can be configured by associating result names to result values.

In [ ]:
result_names = []  # leave empty here
result_values = []  # leave empty here
datalog_results = []  # leave empty here

# Load class names (folder names) from Logs folder
log_dirs = os.listdir("../Logs/")
print("available classes = ", log_dirs)

# For each class (folder), load all data (files in the folder)
datalogs = []
datalogs_split_by_class = []
for class_name in log_dirs:
    datalogs_i = os.listdir("../Logs/" + class_name + "/")
    print(class_name, " --> data logs: ", datalogs_i)
    datalogs_split_by_class.append(datalogs_i)
    for datalog_i in datalogs_i:
        datalogs.append("../Logs/" + class_name + "/" + datalog_i)
        datalog_results.append(class_name)
print("All data logs: ", datalogs)

# NOTE: To take advantage of the metaclassifier block, we suggest class values
# to be in separate groups and also try to avoid 0, it as also corresponds to
# MLCX_SRC register unitialized default result value.
# 
# The metaclassifier group values ranges are:
# - group1: [0..3]
# - group2: [4..7]
# - group3: [8..11]
# - group4: [12..15]

# Assign results and values for decision tree 1:
result_names.append(["still", "walk", "fastwalk"])
result_values.append([1, 4, 8])
# Assign results and values for decision tree 2:
result_names.append([])
result_values.append([])
# Assign results and values for decision tree 3:
result_names.append([])
result_values.append([])
# Assign results and values for decision tree 4:
result_names.append([])
result_values.append([])
# Assign results and values for decision tree 5:
result_names.append([])
result_values.append([])
# Assign results and values for decision tree 6:
result_names.append([])
result_values.append([])
# Assign results and values for decision tree 7:
result_names.append([])
result_values.append([])
# Assign results and values for decision tree 8:
result_names.append([])
result_values.append([])

dectree_filenames = []
for i in range(0, 8):
    if result_names[i] == []:
        break
    else:
        dectree_filenames.append(
            os.path.join(current_directory, f"dectree{i+1}.txt"))

n_decision_trees = i
logging.info("n_decision_trees = %d" % (n_decision_trees))

## Step 2: Generate ARFF files
ARFF files are CSV files where the first n lines declare data types of CSV columns.
By editing lines below, we decide MLC configuration in terms of:
* filters
* features
All the features are computed within a defined time window ('window_length').
Other device settings like MLC data rate, sensor data rate and full scale are defined here.
The module 'mlc_configurator.py' is used for this step.

In [ ]:
# device settings
device_name = "LSM6DSOX"                 # list of supported devices available with get_devices()
mlc_odr = "26 Hz"                        # list of allowed values available with get_mlc_odr(device_name)
input_type = "accelerometer+gyroscope"   # list of allowed values available with get_mlc_input_type(device_name)
accelerometer_fs = "2 g"                 # list of allowed values available with get_accelerometer_fs(device_name)
accelerometer_odr = "26 Hz"              # list of allowed values available with get_accelerometer_odr(device_name)
gyroscope_fs = "2000 dps"                # list of allowed values available with get_gyroscope_fs(device_name)
gyroscope_odr = "26 Hz"                  # list of allowed values available with get_gyroscope_odr(device_name)            
window_length = 52                       # Window length (supported values: from 1 to 255)

# filters configuration
# the list of available filters can be obtained with get_filter_names(input_type)
filters_list = [
    MLCFilter(filter_id="filter_1", name="HP_Acc_XYZ"),
    MLCFilter(filter_id="filter_2", name="BP_Acc_XYZ",   coef_a2=-0.5, coef_a3=0.5,  coef_gain=1.0),
    MLCFilter(filter_id="filter_3", name="IIR1_Acc_V", coef_b1=1.0,  coef_b2=0.25, coef_a2=0.75),
    MLCFilter(filter_id="filter_4", name="IIR2_Acc_V2", coef_b1=0.59, coef_b2=1.19, coef_b3=0.59, coef_a2=-1.01, coef_a3=0.36),
]

# features configuration
# full list of available features can be obtained with get_feature_names(device_name)
# full list of available inputs can be obtained with get_mlc_inputs(input_type)
features_list = [
    MLCFeature(name="MEAN",                   input="Acc_X"),
    MLCFeature(name="VARIANCE",               input="Acc_Y"),
    MLCFeature(name="ENERGY",                 input="Acc_Z"),
    MLCFeature(name="MINIMUM",                input="Acc_V"),
    MLCFeature(name="MAXIMUM",                input="Acc_V2"),
    MLCFeature(name="PEAK_TO_PEAK",           input="Acc_X"),
    MLCFeature(name="ZERO_CROSSING",          input="Acc_Y",  threshold=0.5),
    MLCFeature(name="POSITIVE_ZERO_CROSSING", input="Acc_Z",  threshold=0.5),
    MLCFeature(name="NEGATIVE_ZERO_CROSSING", input="Acc_V",  threshold=0.5),
    MLCFeature(name="PEAK_DETECTOR",          input="Acc_V2", threshold=0.5),
    MLCFeature(name="POSITIVE_PEAK_DETECTOR", input="Acc_X",  threshold=0.5),
    MLCFeature(name="NEGATIVE_PEAK_DETECTOR", input="Acc_Y",  threshold=0.5),
    # Some examples of features computed on filtered inputs:
    MLCFeature(name="MEAN",                   input="Acc_X_filter_1"),
    MLCFeature(name="VARIANCE",               input="Acc_Y_filter_2"),
    MLCFeature(name="ENERGY",                 input="Acc_Z_filter_2"),
    MLCFeature(name="MINIMUM",                input="Acc_V_filter_3"),
    MLCFeature(name="MAXIMUM",                input="Acc_V2_filter_4"),
]

arff_generator(
    device_name,
    datalogs,
    datalog_results,
    mlc_odr,
    input_type,
    accelerometer_fs,
    accelerometer_odr,
    gyroscope_fs,
    gyroscope_odr,
    n_decision_trees,
    window_length,
    filters_list,
    features_list,
    arff_filename,
    current_directory,
)

## Step 3: Decision Tree generation
The module 'decision_tree_generator.py' is used to generate decision trees through sklearn. 

If you have generated the decision tree with external tools, you can skip this section and manually copy the decision tree files in the current directory using: 
* from shutil import copyfile
* source_0 = "PathToYourDecisionTreeFile"
* copyfile(source_0, dectree_filenames[0])

In [ ]:
if (n_decision_trees == 1):
    dectree_accuracy, dectree_nodes = generate_decision_tree(
        arff_filename,
        dectree_filenames[0],
    )
else:
    for i in range(n_decision_trees) :
        arff_filename_i = f"{arff_filename}{i+1}"
        generate_arff_subset(
            arff_filename,
            arff_filename_i,
            result_names[i]
        )
        logging.info(f"\n# Decision Tree {i+1}:")
        dectree_accuracy, dectree_nodes = generate_decision_tree(
            arff_filename_i,
            dectree_filenames[i],
        )


## Step 4: Generation of .ucf file
Settings and files from 'Step 2' and 'Step 3' are used in 'Step 4' to generate the MLC configuration in a .ucf file (a text file containing a sequence of register addresses and corresponding values). 
Meta classifiers, which are filters on the outputs of the decision trees, can be configured here. 
The module 'mlc_configurator.py' is used for this step.

In [ ]:
# Meta-classifiers
# metaclassifierX_values contains the end counter values of the meta classifier associated to the decision tree 'X'
# 4 end counter values are available in LSM6DSOX (the first 4 values in "metaclasifierX_values")
# 8 end counter values are available in LSM6DSRX/ISM330DHCX (the 8 values in "metaclasifierX_values")
# values allowed for end counters are from 0 to 14
metaclassifier_values = [
    "2,2,2,0,0,0,0,0",  # metaclassifier1_values
    "0,0,0,0,0,0,0,0",  # metaclassifier2_values
    "0,0,0,0,0,0,0,0",  # metaclassifier3_values
    "0,0,0,0,0,0,0,0",  # metaclassifier4_values
    "0,0,0,0,0,0,0,0",  # metaclassifier5_values
    "0,0,0,0,0,0,0,0",  # metaclassifier6_values
    "0,0,0,0,0,0,0,0",  # metaclassifier7_values
    "0,0,0,0,0,0,0,0",  # metaclassifier8_values
]

ucf_generator(
    device_name,
    arff_filename,
    dectree_filenames,
    result_names,
    result_values,
    metaclassifier_values,
    ucf_filename,
    current_directory,
)

## (Optional) Generate header file from .ucf file

This cell generates a .h header file after the .ucf file has been generated for use with the STMicroelectronics standard C drivers for MLC configuration without need for using GUI tools.

In [ ]:
h_generator(ucf_filename, h_filename)

## (Optional) Testing decision tree
This section allows to test the .arff file on the decision tree.
The .arff file can be generated in Step2 after selecting data patterns (in Step1) and Features (in Step2). 
The module 'mlc_test.py' is used for this step.

In [ ]:
# Testing
testresults_filename = os.path.join(current_directory, "test_results.txt")
test_arff_on_decisiontree(
    dectree_filenames[0],
    arff_filename,
    testresults_filename,
    validate_results=True,
)

# show test results:
with open(testresults_filename, "r") as f:
    logging.info(f.read())